# Auswertung Adliswil

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
path = "All_Datas/daten_adliswil.csv"
df = pd.read_csv(path)

In [7]:
path2 = "All_Datas/Parlamentarier_Adliswil.csv"
parlamentarier_df = pd.read_csv(path2)

## 1. Welche Parteien reichen die häufigsten Vorstösse ein?

In [12]:
df['ID-Vorstoss'] = df.reset_index().index + 1

In [13]:
df

,Titel,Geschaeftsart,Status,Datum,Namen,Namen-neu,ID-Vorstoss
0,Städtische Beteiligung an Unternehmungen,Postulat,Abgeschrieben,2014-11-21,[' Senn Mario'],['Senn Mario'],1
1,ZKB Jubiläums-Dividende,Interpellation,Beantwortet,2019-11-06,"[' Mäder Gabriel', ' Schanz Simon', '', 'Stefan Neubert (Mitunterzeichner/in)\n Anke Würl (Mitunterzeichner/in)\n Martin Rüttimann (Mitunterzeichner/in)\n Walter Uebersax (Erstunterzeichner/in)', '', '', '']","['Mäder Gabriel', 'Schanz Simon', 'Stefan Neubert', 'Anke Würl', 'Martin Rüttimann', 'Walter Uebersax']",2
2,Zwischennutzung Stadthausareal,Postulat,Überwiesen,2011-09-07,[' Clesle Hanspeter'],['Clesle Hanspeter'],3
3,Anschluss des Quartiers Sonnenberg an den ÖV,Postulat,Abgelehnt,2022-12-15,"[' Muthuthamby Kannathasan', ' Acar Sait']","['Muthuthamby Kannathasan', 'Acar Sait']",4
4,Berücksichtigung von lokalen Handwerker u. Unternehmen bei Beauftragungen durch die Stadt Adliswil,Interpellation,Beantwortet,2022-12-16,[' Muthuthamby Kannathasan'],['Muthuthamby Kannathasan'],5
5,Sichere Schulwege im Soodstrassenbereich,Interpellation,Beantwortet,2011-09-21,[' Baldegger Harry'],['Baldegger Harry'],6
6,Geplante Verkehrsentwicklung,Interpellation,Beantwortet,2019-10-02,"[' Baldegger Harry', ' Mäder Gabriel', ' Melliger Heinz', '', 'Stefan Neubert (Erstunterzeichner/in)\n Anke Würl (Mitunterzeichner/in)\n Walter Uebersax (Mitunterzeichner/in)', '', '']","['Baldegger Harry', 'Mäder Gabriel', 'Melliger Heinz', 'Stefan Neubert', 'Anke Würl', 'Walter Uebersax']",7
7,Reorganisation der Schule Adliswil,Interpellation,Beantwortet,2014-11-17,"['Fredi Morf (keine Funktion)\n Heidi Jucker (keine Funktion)', '']","['Fredi Morf', 'Heidi Jucker']",8
8,Elternmitwirkung bei der Schülerzuteilung,Postulat,Erledigt,2014-11-06,[' Senn Mario'],['Senn Mario'],9
9,Sauberkeit im öffentlichen Raum,Anfrage,Beantwortet,2016-02-10,['Dr. Liedtke Wolfgang'],['Liedtke Wolfgang'],10
